In [1]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import random
from matplotlib import pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from functions import *
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from copy import deepcopy as dp
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
import os
from keras import backend as K 
import optuna
from optuna.integration import KerasPruningCallback
from optuna.testing.integration import DeterministicPruner
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
def gen_model(hidden, drop, vocab_size, embed):
    model = Sequential()
    model.add(Embedding(vocab_size, 20, input_length = embed, mask_zero = True))
    model.add(LSTM(hidden))
    model.add(Dropout(drop))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    return model

In [3]:
import warnings
warnings.filterwarnings('ignore')
optuna.logging.enable_default_handler()

In [4]:
data, w_i, i_d = load_pkl(r'cleaned_corpus/augmented_model_input.pkl')
test_a, train_a = load_pkl(r'cleaned_corpus/augmented_ans.pkl')
test, train = data

sets = list(zip(train, train_a))
random.shuffle(sets)
hate = [text for text in sets if np.argmax(text[1]) == 0]
off = [text for text in sets if np.argmax(text[1]) == 1]
nei = [text for text in sets if np.argmax(text[1]) == 2]
train_set = []
valid_set = []
for i in [hate, off, nei]:
    random.shuffle(i)
    train_set.extend(i[round(len(i)*0.2):])
    valid_set.extend(i[:round(len(i)*0.2)])

In [ ]:
random.shuffle(train_set)
train_X, train_y = zip(*train_set)
valid_X, valid_y = zip(*valid_set)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, min_delta=0.005)
mcp_save = ModelCheckpoint('best_lstm.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
vocab_size = len(w_i)
embed_size = len(train_X[0])

In [ ]:
def objective(trial):
    random.shuffle(train_set)
    train_X, train_y = zip(*train_set)
    params = {
    'lr': trial.suggest_loguniform('lr', 1e-3, 1e-1),
    'hidden': trial.suggest_categorical('hidden', list(range(1, 200, 10))),
    'drop': trial.suggest_uniform('drop', 0.1, 1),
    'batch': trial.suggest_categorical('batch', [10, 30, 50, 90, 135]),
    }
    adam = Adam(lr=params['lr'])
    model = gen_model(params['hidden'], params['drop'], vocab_size= vocab_size, embed=embed_size) 
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[categorical_accuracy])
    history = model.fit(np.array(train_X), np.array(train_y), batch_size=params['batch'], 
                        validation_data = (np.array(valid_X), np.array(valid_y)), 
                        epochs=10, shuffle=True, verbose = 0,
                        callbacks = [es, mcp_save]#, KerasPruningCallback(trial, 'val_acc')]
                       )
    model = load_model('best_lstm.hdf5')
    pred = np.argmax(model.predict(np.array(valid_X)), axis = 1)
    true = np.argmax(valid_y, axis = 1)
    score = f1_score(y_true=true, y_pred=pred, average = 'macro')
    backend.clear_session()
    history
    return 1.0-score

In [ ]:
from keras.backend import tensorflow_backend as backend

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=300, n_jobs=1, verb_pace = 50)

Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00007: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00006: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00009: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00008: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00004: early s

[I 2019-02-27 08:41:43,166] Finished trial 50 / 300 Current best value is 0.11855522931566398 with parameters: {'batch': 50, 'lr': 0.013323570817940833, 'drop': 0.32433889180794917, 'hidden': 181}.


Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00010: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00006: early s

[I 2019-02-27 09:51:24,025] Finished trial 100 / 300 Current best value is 0.11855522931566398 with parameters: {'batch': 50, 'lr': 0.013323570817940833, 'drop': 0.32433889180794917, 'hidden': 181}.


Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00006: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early s

[I 2019-02-27 11:13:35,595] Finished trial 150 / 300 Current best value is 0.11482888838591243 with parameters: {'batch': 30, 'lr': 0.013071298130398291, 'drop': 0.6349548213958957, 'hidden': 171}.


Epoch 00003: early stopping
Epoch 00008: early stopping
Epoch 00010: early stopping
Epoch 00010: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00006: early stopping
Epoch 00005: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00004: early s

[I 2019-02-27 12:34:54,307] Finished trial 200 / 300 Current best value is 0.11482888838591243 with parameters: {'batch': 30, 'lr': 0.013071298130398291, 'drop': 0.6349548213958957, 'hidden': 171}.


Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00009: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping


In [ ]:
save_pkl('tune_result.pkl', study)

In [2]:
best = {'batch': 30, 'lr': 0.013071298130398291, 'drop': 0.6349548213958957, 'hidden': 171}
save_pkl('tuned_parameters/lstm_augmented.pkl', best)